In [1]:
import pandas as pd 
from textblob import TextBlob
import re
import nltk
#nltk.download('words')
words = set(nltk.corpus.words.words())
import sys
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

**1. Combining the Tweet data**

In [2]:


# Reading in October data 
October_26 = pd.read_excel('/Users/jacobkuhlman/Desktop/BITCOIN DATA FINAL/Combined Data/October_26_Data.xlsx')
October_27 = pd.read_excel('/Users/jacobkuhlman/Desktop/BITCOIN DATA FINAL/Combined Data/October_27_Data.xlsx')
October_28 = pd.read_excel('/Users/jacobkuhlman/Desktop/BITCOIN DATA FINAL/Combined Data/October_28_Data.xlsx')
October_29 = pd.read_excel('/Users/jacobkuhlman/Desktop/BITCOIN DATA FINAL/Combined Data/October_29_Data.xlsx')
October_30 = pd.read_excel('/Users/jacobkuhlman/Desktop/BITCOIN DATA FINAL/Combined Data/October_30_Data.xlsx')
October_31 = pd.read_excel('/Users/jacobkuhlman/Desktop/BITCOIN DATA FINAL/Combined Data/October_31_Data.xlsx')

# Reading in November data 
November_1 = pd.read_excel('/Users/jacobkuhlman/Desktop/BITCOIN DATA FINAL/Combined Data/November_1_Data.xlsx')
November_2 = pd.read_excel('/Users/jacobkuhlman/Desktop/BITCOIN DATA FINAL/Combined Data/November_2_Data.xlsx')
November_3 = pd.read_excel('/Users/jacobkuhlman/Desktop/BITCOIN DATA FINAL/Combined Data/November_3_Data.xlsx')
November_4 = pd.read_excel('/Users/jacobkuhlman/Desktop/BITCOIN DATA FINAL/Combined Data/November_4_Data.xlsx')
November_5 = pd.read_excel('/Users/jacobkuhlman/Desktop/BITCOIN DATA FINAL/Combined Data/November_5_Data.xlsx')
November_6 = pd.read_excel('/Users/jacobkuhlman/Desktop/BITCOIN DATA FINAL/Combined Data/November_6_Data.xlsx')
November_7 = pd.read_excel('/Users/jacobkuhlman/Desktop/BITCOIN DATA FINAL/Combined Data/November_7_Data.xlsx')
November_8 = pd.read_excel('/Users/jacobkuhlman/Desktop/BITCOIN DATA FINAL/Combined Data/November_8_Data.xlsx')

In [3]:
Combined_Data = pd.concat([November_1,November_2, November_3, November_4, November_5, November_6, November_7,November_8,
                            October_26, October_27, October_28, October_29, October_30, October_31], axis=0)

**2. Functions used in the notebook**

In [4]:
# Function for renamming columns 
def rename_column(oldName, newName, df):
    New = df.rename(columns={oldName: newName})
    return New

# Function for dropping columns 
def drop(df, column):
    df = df.drop([column], axis=1)
    return df 

# Function for cleaning the tweets 
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
     #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ").replace("RT", " ").replace(":", " ") #Remove hashtag sign but keep the text
    #tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
         #if w.lower() in words or not w.isalpha())
    return tweet

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


**3. Data manipulation**

In [ ]:
# Dropping unnamed column 
Combined_Data = Combined_Data.loc[:, ~Combined_Data.columns.str.contains('^Unnamed')]

# Cleaing the tweets: Taking @ out, taking links out etc. 
Combined_Data['Clean_Tweet'] = Combined_Data['Tweet'].map(lambda x: cleaner(x))

Combined_Data['Clean_Tweet'] = Combined_Data['Clean_Tweet']

# Making the cleaned tweet column all lower case 
Combined_Data["Clean_Tweet"] = Combined_Data["Clean_Tweet"].str.lower()



# Remove punctuation from the tweet column 
Combined_Data["Clean_Tweet"] = Combined_Data["Clean_Tweet"].str.replace('[^\w\s]','')

# Remove emojis from tweet column 
Combined_Data["Clean_Tweet"] = Combined_Data["Clean_Tweet"].apply(lambda x: remove_emoji(x))

# Changing the order of columns in the dataframe 
Combined_Data = Combined_Data[['Date', 'ID', 'Author ID', 'Tweet', 'Clean_Tweet', 'public_metrics', 'entities', 'Language']]


# creating new columns from the 'public_metrics' column that was collected from twitter 
Combined_Data[['retweet_count', 'reply_count', 'like_count', 'quote_count']] = Combined_Data['public_metrics'].str.split(',', 4, expand=True)


# getting only the numbers extracted for these columns 
Combined_Data['retweet_count'] = Combined_Data['retweet_count'].str[17:]
Combined_Data['reply_count'] = Combined_Data['reply_count'].str[15:]
Combined_Data['like_count'] = Combined_Data['like_count'].str[14:]

# dropping un-needed columns 
Combined_Data = drop(Combined_Data, 'entities')
Combined_Data = drop(Combined_Data, 'quote_count')
Combined_Data = drop(Combined_Data, 'public_metrics')
Combined_Data = drop(Combined_Data, 'Language')

**4. Converting data to excel file**

In [18]:
Combined_Data.to_excel("Combined_Data.xlsx")